In [ ]:
# 11/7/2021-1
# https://www.tensorflow.org/tutorials/load_data/images
from __future__ import absolute_import
from IPython.display import clear_output
import glob, os, shutil, sys, uuid
from os.path import *
from time import sleep
from TarfileFunctions import *
from BashColors import C

contentPath = os.getcwd()
cv2Path = join(contentPath, 'CV2Images')

if exists(cv2Path):
    shutil.rmtree(cv2Path)
    sleep(0.05)
if not exists(cv2Path):
    tff.extractTarfiles('All_CV2_Images.tar.gz')
    sleep(0.05)

try:
    import tensorflow as tf
    from tensorflow import keras
except Exception as err:
    print(err)
    %pip install tensorflow
    
import numpy as np

try: 
    from matplotlib import pyplot as plt
except Exception as err:
        %pip install matplotlib
        from matplotlib import pyplot as plt

try:
    import cv2
except Exception as err:
    %pip install opencv-python-headless # works.
    import cv2
from CV2_Utils import *
    
os.chdir(contentPath)
clear_output()
import tensorflow as tf
print(f'tf version: {C.IBlue}{tf.__version__}{C.ColorOff}')
print(f'cwd: {C.IBlue}{os.getcwd()}{C.ColorOff}')

In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory
train_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgba',
    validation_split=0.2,
    subset="training",
    seed=456,
    image_size=(224, 224),
    batch_size=4)
print()
val_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgba',
    validation_split=0.2,
    subset="validation",
    seed=456,
    image_size=(224, 224),
    batch_size=4)
print()


In [ ]:
for image, labels in train_ds:
    print(f'image.shape: {image.shape}')
    print(f'labels.shape: {labels.shape}')
    break

In [ ]:

from tensorflow.keras.models import Sequential, SparseCategoricalCrossentropy
# Define a simple sequential model
def create_model():
    pretrained_model = tf.keras.applications.MobileNet()
    result_before_save = pretrained_model(x)
    decoded = imagenet_labels[np.argsort(result_before_save)[0,::-1][:5]+1]
    print("Result before saving:\n", decoded)
    return pretrained_model

In [ ]:
# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback =tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1)

history=model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=2,
    callbacks=[cp_callback])  # Pass callback to training


print(f'{C.IBlue}')
print(history.history)

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.